In [ ]:
import autograd.numpy as np
import autograd.numpy.random as npr
npr.seed(0)

import pandas as pd
import cst
import pyaldata

import ssm
from ssm.util import find_permutation
from ssm.plots import gradient_cmap, white_to_color_cmap

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

color_names = [
    "windows blue",
    "red",
    "amber",
    "faded green",
    "dusty purple",
    "orange"
    ]

colors = sns.xkcd_palette(color_names)
cmap = gradient_cmap(colors)


# Speficy whether or not to save figures
save_figures = False

In [ ]:
filename = '/home/raeed/data/project-data/smile/cst-gainlag/library/python/Ford_20180618_COCST_TD.mat'
# td = cst.get_cst_dataframe(filename)
td = pyaldata.mat2dataframe(filename,True,'td_cst')

In [ ]:
td['cursor_pos']

In [ ]:
trial_info = td['monkey':'lambda']
cont_sigs = 

In [ ]:
num_states = 2
obs_dim = 2
input_dims = 2
N_iters = 50

data = obs # Treat observations generated above as synthetic data.

## testing the constrained transitions class
hmm = ssm.HMM(num_states, obs_dim, M=input_dims, observations="autoregressive",transitions='standard')

hmm_lls = hmm.fit(obs, inputs=random_input, method="em", num_iters=N_iters, init_method="kmeans") #can also use random for initialization method, which sometimes works better

plt.plot(hmm_lls, label="EM")
plt.plot([0, N_iters], true_ll * np.ones(2), ':k', label="True")
plt.xlabel("EM Iteration")
plt.ylabel("Log Probability")
plt.legend(loc="lower right")
plt.show()